### 목적/Import
+ 목적 : NPL 모델을 이용한 리뷰 SCORE 예측(1~5점)
+ 목차
  1. [x] 데이터 자료준비
    + [x] BigQuery(review, 4 channenl)
    + [x] tokenzie(Konlpy)
    + [x] padding (length standarzation)
  2. [ ] modeling
    + [x] 1D-CNN
    + [ ] LSTM?
    + [ ] huggingface?
  3. [ ] hyperparameter-tunning : wandb 
    + [ ] single
    + [ ] sweept
  4. [ ] engeneering 
    + [ ] 모델저장 : .h5 파일 저장
    + [ ] 모델에 대한 google storage저장
    + [ ] 모델 호출을 위한 전처리 코드(=API)


In [ ]:
# !pip install wandb -qqq
# !pip install konlpy -qqq
!pip install -qqq transformers datasets

     |████████████████████████████████| 3.1 MB 34.2 MB/s 
     |████████████████████████████████| 290 kB 88.2 MB/s 
     |████████████████████████████████| 596 kB 74.8 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
     |████████████████████████████████| 3.3 MB 47.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 132 kB 83.4 MB/s 
     |████████████████████████████████| 243 kB 62.5 MB/s 
     |████████████████████████████████| 160 kB 52.5 MB/s 
     |████████████████████████████████| 192 kB 58.3 MB/s 
     |████████████████████████████████| 271 kB 72.7 MB/s 


In [ ]:
from konlpy.tag import Okt
okt = Okt()

import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OrdinalEncoder

import numpy as np
import pandas as pd

import wandb
from wandb.keras import WandbCallback

from google.cloud import bigquery
import pandas as pd
import numpy as np

client = bigquery.Client(project = 'thermal-rain-234004')

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### 1.데이터 준비

#### BigQuery

In [ ]:
##########
# 자료 불러오기
###

# BigQuery
sql = """SELECT *
          FROM `thermal-rain-234004.review.review_all` A
          where SCORE <> 0 """
df1 = client.query(sql).to_dataframe()

# 데이터 보정
ord_enc = OrdinalEncoder()
df = df1.copy()
df['label'] = ord_enc.fit_transform(df[['SCORE']])
df['REVIEW'] = df['REVIEW'].str.replace("\n"," ")
df['REVIEW'] = df['REVIEW'].str.replace("\r"," ")
df['REVIEW'] = df['REVIEW'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

df = df[['REVIEW', 'label']]

In [ ]:
df1.tail(2)

,USER,CHANNEL,PRODUCT,REVIEW,SCORE,DATE,GUBUN
3328,신인**************,카카오 쇼핑하기,[매일유업] 허쉬 오리지널 /우유속에 딸기/바나나는원래하얗다 190ml 24팩 3종 택1,맛있어요\r\n싸게 잘 사서 부담없이 잘 먹어요,5,2021-11-05 02:55:15+00:00,리뷰
3329,경*,카카오 쇼핑하기,[매일유업] 허쉬 오리지널 /우유속에 딸기/바나나는원래하얗다 190ml 24팩 3종 택1,배송 빨라요,5,2021-11-05 13:23:04+00:00,리뷰


In [ ]:
df1.groupby('CHANNEL').count()

,USER,PRODUCT,REVIEW,SCORE,DATE,GUBUN
CHANNEL,,,,,,
G마켓,34,34,34,34,34,34
kakaomakers,2328,2328,2328,2328,2328,2328
카카오 선물하기,11,11,11,11,11,11
카카오 쇼핑하기,957,957,957,957,957,957


#### split + tokenzie(Konlpy)

In [ ]:
##########
# train/test Set 분류
###
train_data, test_data = train_test_split(df, test_size = 0.25, random_state = 42)

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_train = []
for sentence in train_data['REVIEW']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

X_test = []
for sentence in test_data['REVIEW']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

#### padding

In [ ]:
##########
# pedding
###


threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [ ]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 1152


In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 95.63476171405686


In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
y_train = train_data['label'].to_list()
y_train = np.asarray(y_train).astype('uint8')

y_test = test_data['label'].to_list()
y_test = np.asarray(y_test).astype('uint8')

In [ ]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0, 215,  31,  22,  40,  30, 114, 142,
       498,  27,   1,  66], dtype=int32)

In [ ]:
y_train[0]

4

### 2.modeling

#### 1D-CNN(vanilla)

In [ ]:
# Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Conv1D(32, 5, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(config.layer_2, activation=config.activation_2),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Dense(5, activation='sigmoid')
    ])

model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          36864     
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

In [ ]:
history = model.fit(x=X_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(X_test, y_test))

Epoch 1/8
79/79 [==============================] - 1s 6ms/step - loss: 1.3824 - accuracy: 0.7821 - val_loss: 1.2234 - val_accuracy: 0.8031
Epoch 2/8
79/79 [==============================] - 1s 9ms/step - loss: 1.1218 - accuracy: 0.7994 - val_loss: 1.0287 - val_accuracy: 0.8031
Epoch 3/8
79/79 [==============================] - 0s 4ms/step - loss: 0.9723 - accuracy: 0.7994 - val_loss: 0.9128 - val_accuracy: 0.8031
Epoch 4/8
79/79 [==============================] - 0s 4ms/step - loss: 0.8830 - accuracy: 0.7994 - val_loss: 0.8413 - val_accuracy: 0.8031
Epoch 5/8
79/79 [==============================] - 0s 4ms/step - loss: 0.8286 - accuracy: 0.7994 - val_loss: 0.7984 - val_accuracy: 0.8031
Epoch 6/8
79/79 [==============================] - 0s 4ms/step - loss: 0.7948 - accuracy: 0.7994 - val_loss: 0.7715 - val_accuracy: 0.8031
Epoch 7/8
79/79 [==============================] - 0s 4ms/step - loss: 0.7742 - accuracy: 0.7994 - val_loss: 0.7536 - val_accuracy: 0.8031
Epoch 8/8
79/79 [==========

#### LSTM

#### hugging-face

In [ ]:
# from transformers import pipeline
# classifier = pipeline('sentiment-analysis')

## 3.parameter tuning : single

In [ ]:
  wandb.init(
      project="NLP-practics_v2",
      # Set entity to specify your username or team name
      # ex: entity="carey",
      config={
          "layer_1": 128,
          "activation_1": "relu",

          "layer_2": 10,
          "activation_2": "softmax",

          "dropout": 0.05,
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 8,
          "batch_size": 32
      })
  config = wandb.config

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Conv1D(32, 5, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(config.layer_2, activation=config.activation_2),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Dense(2, activation='sigmoid')
    ])

model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

In [ ]:
# vocab_size

In [ ]:
  history = model.fit(x=X_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(X_test, y_test),
                      # WandbCallback auto-saves all metrics from model.fit()
                      # callbacks=[WandbCallback()]
                      )

Epoch 1/8


InvalidArgumentError: ignored

In [ ]:
# model.summary()

In [ ]:
y_test[0]

0

## 4.parameter tunning : sweept

In [ ]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'layers': {
            'values': [32, 64, 96, 128, 256]
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4]
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'optimizer': {
            'values': ['adam', 'rmsprop']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [ ]:
def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
          "layers": 32,
          "dropout_rate": 0.1,
          "layer1_activation": 'relu',
          "optimizer": "adam",
          "learning_rate" : 0.1
    }
    wandb.init(project = 'NLP-practics_v2',
               config=config_defaults,
               magic=True)
    config = wandb.config

    # Get the data
    x_train, x_test = X_train,  X_test
    y_train, y_test = y_train,  y_test

    # Build a model
    model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(vocab_size, 32),
      tf.keras.layers.Dropout(wandb.config.dropout),
      tf.keras.layers.Conv1D(32, 5, strides=1, padding='valid', activation=layer1_activation),
      tf.keras.layers.GlobalMaxPooling1D(),
      tf.keras.layers.Dense(wandb.config.layers, activation=wandb.config.activation_2),
      tf.keras.layers.Dropout(wandb.config.dropout_rate),
      tf.keras.layers.Dense(2, activation='sigmoid')
      ])
    if config.optimizer =='rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
    else:
      opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    model.compile(optimizer=wandb.config.optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5,
                  validation_data=(x_test, y_test))


In [ ]:
import wandb
sweep_id = wandb.sweep(sweep_config, project= 'NLP-practics_v2')

Create sweep with ID: 131a430a
Sweep URL: https://wandb.ai/xyzmagic/NLP-practics_v2/sweeps/131a430a


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: u57zstxo with config:
wandb: 	dropout_rate: 0.1
wandb: 	layer1_activation: relu
wandb: 	layers: 32
wandb: 	learning_rate: 0.1
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: ERROR Run u57zstxo errored: UnboundLocalError("local variable 'y_train' referenced before assignment")
wandb: Agent Starting Run: tzrw06gz with config:
wandb: 	dropout_rate: 0.1
wandb: 	layer1_activation: relu
wandb: 	layers: 32
wandb: 	learning_rate: 0.1
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: ERROR Run tzrw06gz errored: UnboundLocalError("local variable 'y_train' referenced before assignment")
wandb: Agent Starting Run: im90fhac with config:
wandb: 	dropout_rate: 0.1
wandb: 	layer1_activation: relu
wandb: 	layers: 32
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: ERROR Run im90fhac errored: UnboundLocalError("local variable 'y_train' referenced before assignment")
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
